> Algorithm

1. $f_{trimed} \leftarrow Update(f,f_{Ebayes})$

2. $Construct G_{trimed} = \{ f_{trimed}, \mathcal{W}_{n \times n} , \mathcal{V} \}$

3. $\hat{G} \leftarrow STGCN(Construct G_{trimed} , lag, epoch)$

4. from $\hat{G}$ obtain $\hat{f}$

# Info

**data_dict**

- `edges` = ? $\times$ 2
    - It depends on connection of each node of dataset
- `node_ids` = N
- `FX` = TN $\times$ 1

# Missing

## padding

- `mindex` = 입력값, 리스트 자료형
- `f` = TN $\times$ 1, 리스트 자료형
    - 보간법 이용 후 결측값 뒤, 앞으로 채우고 리스트화하여 `FX` 값으로 출력
- `lags` = up to writer's choice
- `T` = Time
- `N` = Node

:::{.callout-tip}
- linear: 선형 보간법(기본값)
- time: 시계열 데이터를 위한 선형 보간법
- index: 인덱스를 사용하여 선형 보간법
- polynomial: 다항식 보간법
- spline: 스플라인 보간법
- barycentric: 바리센트릭 보간법
- krogh: Krogh 보간법
- piecewise_polynomial: 조각 다항식 보간법
- pchip: PCHIP(Piecewise Cubic Hermite Interpolating Polynomial) 보간법
- akima: Akima 보간법
:::

:::{.callout-note}
`fillna(method='bfill')`

이 코드는 Pandas 라이브러리를 사용하여 데이터프레임의 결측값을 뒤 방향으로(backward) 채우는(fill) 메소드입니다.

'bfill'은 'backward fill'을 의미하며, 이 메소드는 결측값(missing value)을 다음으로 나타나는(non-missing) 값으로 대체합니다. 즉, 각 결측값은 그 이후에 오는 첫 번째 비 결측값으로 대체됩니다.
:::

:::{.callout-note}
`fillna(method='ffill')`

이 코드는 Pandas 라이브러리를 사용하여 데이터프레임의 결측값을 앞 방향으로(forward) 채우는(fill) 메소드입니다.

'ffill'은 'forward fill'을 의미하며, 이 메소드는 결측값을 이전에 나타난(non-missing) 값으로 대체합니다. 즉, 각 결측값은 그 이전에 나타난 마지막 비 결측값으로 대체됩니다.
:::

## rand_mindex

- `missing_count` = mrate $\times$ Time
- `mindex` = Index of random values
    - block에서 시뮬레이션 돌리려면 나중에 트레이닝에서 missing 뽑을 거니까 트레이닝$\times$0.8보다 작은 값으로 block 설정해야 함

# miss

In [9]:
mindex= [[],[],list(range(1,3))]

In [11]:
for i,m in enumerate(mindex): 
        print(m)

[]
[]
[1, 2]


In [10]:
for i,m in enumerate(mindex): 
        print(i)

0
1
2


```python
for i,m in enumerate(mindex): 
        f[m,i] = np.nan
```

- mindex는 리스트 자료형으로, node 수 만큼 있으며, 그 node 안에 mindex값이 존재하게끔
    - `m` = 각 리스트, node 수 만큼 나옴
    - `i` = node 번호 순 0부터

# leanrers

## make_Psi

- `W` = T $\times$ T
    - 대각선빼고 모두 1인, 모두 연결되었다고 가정하는 인접행렬(adjacency matrix)
- `d` = sum of each row data at `W`
    - 2차원 $\to$ 1차원
    - T $\times$ 1
- `D` = 대각선 원소가 `d`값을 가진 2차원 행렬로 변환
    - T $\times$ T
- `L` = $D^{1/2} (D-W)D^{1/2}$
- `lamb`, `Psi` = result of eigenvaluede composition at `L`
    - `lamb` = T,
    - `Psi` = T $\times$ T

:::{.callout-important}
$L = D-W$

$D$ is a diagonal degree matrix.

$W$ is a weight matrix.

$\tilde{L} = D^{-1/2}LD^{-1/2}$

$= D^{-1/2}(D-W)D^{-1/2}$

$= D^{-1/2}DD^{-1/2}-D^{-1/2}WD^{-1/2}$

$= I-D^{-1/2}WD^{-1/2}$

$I$ is an identity matrix.

$\tilde{L}$ nomalizes weights using $D^{-1/2}WD^{-1/2}$ by degree of each node and retain the information of graph structiure.

$0 < L< 1$
:::

## trim

- `f` = T $\times$ N $\to$ 벡터로 변환, ?$\times$1

아래와 같이

In [20]:
f = np.array([1,2]);f.shape

(2,)

In [21]:
if len(f.shape)==1: f = f.reshape(-1,1)

In [22]:
f.shape

(2, 1)

- `T` = Time
- `N` = Node
- `Psi` = Psi from make_Psi class
    - T $\times$ T
- `fbar` = trans(Psi) $\times$ `f` = T $\times$ N
    - `Psi` = T$\times$ T
    - `f` = T $\times$ N
- `fbar_threshed` = fbar에서 node 수 대로 ebayes하여 step function을 detect 하여 행별로 stack 한 결과
    - T $\times$ N
- `fhat` = Psi $\times$ fbar_threshed = T $\times$ N
    - Psi = T $\times$T
    - fbar_threshed = T $\times$ N

## update_from_freq_domain

- `signal` = input
- `signal_trimed` = trim(signal) = T$\times$N

`missing_index`를 입력으로 바꿔서 `signal_trimed`로 결측 index의 값만 imputation

## RecurrentGCN

:::{.callout-tip}
GConvGRU를 사용하였으며, Structured Sequence Modeling with Graph Convolutional Recurrent Networks 논문에서 제안된 모델

이 논문은 그래프 합성곱 신경망 (Graph Convolutional Neural Network, GCN)을 이용한 시퀀스 모델링을 위해 제안된 구조인 그래프 합성곱 순환 신경망 (Graph Convolutional Recurrent Neural Network, GCRN)에 대한 연구입니다.

GCRN은 시계열 데이터를 처리하기 위해 순환 신경망 (Recurrent Neural Network, RNN)과 같은 장기 의존성(long-term dependency)을 고려하는 방법으로 그래프 합성곱 신경망(GCN)을 확장합니다. 이를 위해 GCRN은 각 시간 스텝에서 그래프 상의 노드 간 상호 작용을 잘 나타낼 수 있는 새로운 그래프 합성곱 셀(GCN cell)을 도입합니다. 이 셀은 체비셰폴리노미알 필터 (Chebyshev Polynomial Filter)를 사용하여 그래프 신호의 재귀적인 확장을 수행하며, 게이트 메카니즘(Gate Mechanism)을 통해 순환 구조를 적용합니다.

논문에서 실험은 여러 시퀀스 모델링 문제에서 GCRN의 성능을 검증합니다. 실험 결과 GCRN은 기존의 RNN과 GCN을 비롯한 다른 모델들과 비교하여 높은 예측 정확도를 보입니다. 이를 통해 GCRN이 그래프 시계열 데이터 분석 분야에서 유용하게 사용될 수 있다는 가능성을 제시합니다.

Chebyshev polynomial filter는 Laplacian 행렬과 함께 사용하여 그래프 신경망에서 필터링을 수행하는 데 사용됩니다. 이 필터는 임의의 그래프에 대해 일반적으로 적용될 수 있으며, 이전의 고유 벡터와 함께 Laplacian의 K차 필터링을 수행하는 데 사용됩니다. 이 때 K는 Chebyshev 다항식의 차수입니다.
:::

## StgcnLearner

```python
getattr(dataset,'mindex',None)
```

dataset에서 mindex 가져오기

## ITStgcnLearner

## GNARLearner